---
title: "Visualization (Exploring variation)"
author: "Peter Ganong and Maggie Shi"
date: today
date-format: long
format: 
    html:
        echo: true
        code: true
        toc: true
        output: true
engine: jupyter
---

In [ ]:
#| echo: false
import altair as alt
from vega_datasets import data
from palmerpenguins import load_penguins
from plotnine import *
from plotnine.data import diamonds, mpg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Motivation

## Introduction to the next two lectures

Most of our visualization lectures are based on the University of Washington textbook, but the textbook doesn't have enough material on exploratory data analysis. We therefore are supplementing with the [Data Visualization](https://r4ds.hadley.nz/data-visualize) and [Exploratory Data Analysis](https://r4ds.hadley.nz/eda) material in the R for Data Science textbook (with the code translated to Altair).

* `diamonds` is from "Exploratory Data Analysis"
* `movies` is from the UW textbook 
* `penguins` is from "Data Visualization" 

## What is exploratory data analysis?

Data visualization has two distinct goals

1. **exploration** for you to learn as much as possible
2. **production** for you to teach someone else what you think the key lessons are

How do the modes differ?

* When you are in exploration mode, you will look at lots of patterns and your brain filters out the noise
* Production mode is like putting a cone on your dog. You are deliberately limiting the reader's field of vision such that they see the key messages from the plot *and avoid too many distractions*

The next two lectures are almost entirely about **exploration**. Then, at the end of lecture 5, we will transition to thinking about graphics for production. Lecture 6 will similarly about graphics for production.

Caveat: these modes make the most sense when thinking about *static* visualization. Later on in the course, when we talk about dashboards, this is closer to making interfaces to help readers who don't code explore the data. 


# Categorical variables

## Categorical variables: roadmap 

* introduce `diamonds`
* show table
* show bar graph

## introduce dataset `diamonds`

In [ ]:
from plotnine import *
from plotnine.data import diamonds, mpg
diamonds

## `diamonds` data dictionary
(Accessed by running `?diamonds` in R)
A data frame with 53940 rows and 10 variables:

* `price` - price in US dollars ($326–$18,823)
* `carat`- weight of the diamond (0.2–5.01)
* `cut` - quality of the cut (Fair, Good, Very Good, Premium, Ideal)
* `color` - diamond colour, from D (best) to J (worst)
* `clarity` - a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
* `x` -- length in mm (0–10.74)
* `y` -- width in mm (0–58.9)
* `z` -- depth in mm (0–31.8)
* `depth` -- total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43–79)
* `table` -- width of top of diamond relative to widest point (43–95)

##  `diamonds` 

In [ ]:
#| warning: false
diamonds_cut = diamonds.groupby('cut').size()
diamonds_cut

## Categorical variables


In [ ]:
diamonds_cut = diamonds_cut.reset_index().rename(columns={0:'N'}) # Prepare to plot

alt.Chart(diamonds_cut).mark_bar().encode(
    alt.X('cut'),
    alt.Y('N')
)

## Categorical variables -- summary

* this section is very brief because there's basically only one good way to plot categorical variables with a small number of categories and this is it. 
    * You can use `mark_point()` instead of `mark_bar()`, but overall, there's a clear right answer about how to do this. 
* We include this material mainly to foreshadow the fact that we will do a lot on categorical variables in the next lecture when we get to "Exploring Co-variation"


# Continuous variables

## Roadmap: Continuous variables

* histograms using `movies`
* histograms and density plots using `penguins`
* diamond size (carat)

Remark: The skills are absolutely fundamental and so we will intentionally be a bit repetitive.


## movies dataset

In [ ]:
movies_url = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'

In [ ]:
#| eval: false
movies = pd.read_json(movies_url)

In [ ]:
#| echo: false
movies = pd.read_json('data/movies.json')

## recap scatter plot from lecture 3

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('IMDB_Rating:Q')
)

One question which came up (which is hard to tell from this scatter plot, even with bins) is how many observations are there in each bin

## scatter plot -- N movies per bin

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('count(IMDB_Rating):Q')
)

## scatter plot -- syntax trick
Replace `count(IMDB_Rating)` with `count()` because we aren't using IMDB rating any more.

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('count():Q')
)

## histogram using `mark_bar()`

In [ ]:
hist_rt = alt.Chart(movies_url).mark_bar().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('count():Q')
)
hist_rt

Discussion question: how would you describe the distribution of rotten tomatoes ratings?

::: {.notes .content-visible when-profile="speaker"}
**NOTES:**
*solution: 
main answer: close to uniform.
two additional details are also notable
1) the bottom bin out of 20 has very few movies 
2) there's a gentle shift/trend upward so higher-rated movies are a bit more common than lower-rated movies*
:::


## histogram of IMDB ratings

In [ ]:
hist_imdb = alt.Chart(movies_url).mark_bar().encode(
    alt.X('IMDB_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('count():Q')
)
hist_imdb

## Side-by-side
Discussion question -- compare the two ratings distributions. which is more informative?

::: {.notes .content-visible when-profile="speaker"}
**NOTES:**
*solution: rotten tomatoes. IMDB is too compressed.*
:::


In [ ]:
hist_rt | hist_imdb

## introducing the penguins

In [ ]:
from palmerpenguins import load_penguins
penguins = load_penguins()
display(penguins)

## histogram with steps of 200

In [ ]:
alt.Chart(penguins).mark_bar().encode(
    alt.X('body_mass_g', bin=alt.BinParams(step=200)),
    alt.Y('count()')
)

## histogram `step` parameter

20 vs 200 vs 200


In [ ]:
#| echo: false

plot1 = alt.Chart(penguins).mark_bar().encode(
    alt.X('body_mass_g', bin=alt.BinParams(step=20)),
    alt.Y('count()'))

plot2 = alt.Chart(penguins).mark_bar().encode(
    alt.X('body_mass_g', bin=alt.BinParams(step=200)),
    alt.Y('count()'))

plot3 = alt.Chart(penguins).mark_bar().encode(
    alt.X('body_mass_g', bin=alt.BinParams(step=2000)),
    alt.Y('count()'))

plot1 & plot2 & plot3

Discussion q -- what message comes from each `binwidth` choice?

::: {.notes .content-visible when-profile="speaker"}
**NOTES:**
*solution 
the middle plot is by far the easiest to interpret. The top plot shows the bins in such a thin way that it is hard to see much of anything. 

FYI, this deliberately intended to be a setup for the diamonds density plot coming up in the next subsection because there adding narrower bins is actually quite revealing*
:::

## numeric variable: `transform_density()` 

In [ ]:
alt.Chart(penguins).transform_density(
    'body_mass_g',
    as_=['body_mass_g', 'density']
).mark_area().encode(
    x='body_mass_g:Q',
    y='density:Q'
)

## Back to diamonds, focus on `carat`

In [ ]:
alt.data_transformers.disable_max_rows() # Needed because len(df) > 5000

alt.Chart(diamonds).mark_bar().encode(
    alt.X('carat', bin=alt.Bin(maxbins=10)),
    alt.Y('count()')
)

## Continuous Variables

In [ ]:
#| warning: false
diamonds['bins'] = pd.cut(diamonds['carat'], bins=10)
diamonds.groupby('bins').size()

## Continuous Variables: Typical Values

In [ ]:
diamonds = diamonds.drop('bins', axis=1) # 'Interval' type causes plotting issues 
diamonds_small = diamonds.loc[diamonds['carat'] < 2.1] # Subset to small diamonds

alt.Chart(diamonds_small).mark_bar().encode(
    alt.X('carat', bin=alt.BinParams(step=0.1)),
    alt.Y('count()')
)

## Continuous Variables: Typical Values

In [ ]:
alt.Chart(diamonds_small).mark_bar().encode(
    alt.X('carat', bin=alt.BinParams(step=0.01)),
    alt.Y('count()')
)

Discussion questions 

1. What lessons does this plot teach?
2. What questions does it raise?

::: {.notes .content-visible when-profile="speaker"}
**NOTES:**
*
1. there's a ton of missing mass.
2. Are diamonds being cut down to exact sizes? Are diamond weights being fraudulently inflated? Why are some diamonds cut and not others?*

*ask class how many have been to the Art Institute and see this painting*
:::

## Aside: "A Sunday on La Grande Jatte" by Seurat

![](pictures/seurat.jpg)

::: {.notes .content-visible when-profile="speaker"}
**NOTES:**
*for those of you who have not seen the painting (and perhaps some of you who have seen it), you noticed that it's actually made up of millions of little dots. This is a style known as pointillism. Sort of like pixels before we had computer screens. It's a good example of how you can notice more detail and more interesting detail by looking closely at a painting or a plot.

In addition, there's something else (flip back) that you notice by looking closely which is that none of the people in the painting are looking at each other. They are all in their own worlds, isolated from and not interacting with one another. Obviously this is not an art course, our main point is just that there's huge value to looking closely at the plots that you make*
:::


## Aside: "A Sunday on La Grande Jatte" by Seurat
![](pictures/seurat_zoomed.jpg)



# Unusual numeric values (`diamonds`)
<!-- source: Lecture 5, slides 26-44 -->

## roadmap

* case study 1: `y` dimension in diamonds
    * explore some unusual  values
    * three options for handling unusual values
* case study 2 (next section): cars' gas mileage

## Diamonds: examine unusual values


In [ ]:
diamonds['y'].describe()

## Diamonds: examine unusual values 

In [ ]:
diamonds.loc[(diamonds['y'] < 3) | (diamonds['y'] > 20)] 

## Diamonds: sanity check by comparing to 10 random diamonds

In [ ]:
diamonds.sample(n=10)

## What to do with unusual values?

1. Drop row
2. Code value to `NA`
3. Winsorize value

## Diamonds: option 1 for unusual values: drop

In [ ]:
diamonds_clean = diamonds.loc[(diamonds['y'] >= 3) | (diamonds['y'] <= 20)] 
diamonds_clean

## Diamonds: option 2 for unusual values: missing

In [ ]:
diamonds['y'] = np.where((diamonds['y'] < 3) | (diamonds['y'] > 20), np.nan, diamonds['y'])
diamonds_clean = diamonds.dropna()
diamonds_clean

## Diamonds: option 3 for unusual values: winsorize

In [ ]:
pctile01 = diamonds['y'].quantile(0.01)
pctile99 = diamonds['y'].quantile(0.99)

print(f"1st Percentile: {pctile01}")
print(f"99th Percentile: {pctile99}")

## Diamonds: option 3 for unusual values: winsorize

In [ ]:
diamonds['y_winsor'] = np.where(diamonds['y'] < pctile01, pctile01, 
                                np.where(diamonds['y'] > pctile99, pctile99, diamonds['y']))
diamonds

When is this useful? Income data, test scores, stock returns. Important when you are using procedures where the estimates are sensitive to outliers like computing a mean or running a regression

## how do I know which option to choose?

* make an educated guess by looking at the data as many ways as possible
* you often can ask your data provider... but they will quickly grow impatient so try to answer as many questions as possible yourself

## Diamonds: what should we actually do?

My take (there is often not a ``right'' answer or you won't know the answer without talking to a data provider)

* Rows where `x`, `y`, and `z` are all zero: set to `NA`
* Rows where `y > 20`: winsorize? (hard to know for sure...)

## Summary: handling unusual numeric values 

| Problem | Action |
| ---- | --- | 
| Erroneous row | drop row |
| Erroneous cell | set to NA or winsorize | 

How do I decide which problem I have? Examine unusual values in context of other columns (same row) and other rows (same columns). We will see this again in a future lecture.

How do I decide whether to set to NA or winsorize? Ideally, ask your data provider what's going on with these values. 




# Unusual values case study

## Introducing the `mpg` dataset


In [ ]:
mpg

## Q: Why do some cars have better than typical mileage?


In [ ]:
potential_outliers = mpg.loc[(mpg["hwy"] > 40) | ((mpg["hwy"] > 20) & (mpg["displ"] > 5))]
potential_outliers

Note: calling `geom_point()` more than once!

## Q: Why do some cars have better than typical mileage?


In [ ]:
base = alt.Chart(mpg).mark_point().encode(
         alt.X('displ:Q', title = "Engine size (displ)"),
         alt.Y('hwy:Q', title = "Gas mileage")
    ).properties(
        width=600, 
        height=400 )

outliers = alt.Chart(potential_outliers).mark_point(
    color='red',
    size=100,
    shape='circle'
    ).encode(
        x='displ:Q',
        y='hwy:Q'
    ).properties(
        width=600, 
        height=400)
plot = base + outliers
plot

## Q: Why do some cars have better than typical mileage?


In [ ]:
labels = alt.Chart(potential_outliers).mark_text(
    align='left',
    dx=10,  # Adjust horizontal distance of text from the point
    dy=-5   # Adjust vertical distance of text from the point
).encode(
    alt.X('displ:Q', title = "Engine size (displ)"),
    alt.Y('hwy:Q', title = "Gas mileage"),
    text='model:N'  # Display car_model as the label
).properties(
    width=600, 
    height=400)

plot = base + outliers + labels
plot

## Q: How are there big engines and good mileage? `color`

In [ ]:
alt.Chart(mpg).mark_point(size=100).encode(
    x='displ:Q',  # Quantitative variable for displacement
    y='hwy:Q',    # Quantitative variable for highway mpg
    color='class:N',  # Categorical variable for class
    tooltip=['displ', 'hwy', 'class']  # Optional: tooltip to display values on hover
)

## gas mileage summary

* Question: Why do some cars have better than typical mileage? *(What's going on with these outliers?)*
    * Tools: 
        * identify outliers 
        * `color = class`  
    * Answer: 2-seaters & subcompact 
